# Bosonic Double-well

In [ ]:
using KadanoffBaym
using LinearAlgebra

using PyPlot
using PyCall
qt = pyimport("qutip")
np = pyimport("numpy")
sp = pyimport("scipy")

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=16)

## Model

### Non-Hermitian Hamiltonian:

$$
    \hat{H} = \left(\omega_1 -i\frac{\lambda}{2}\right) a^{\dagger}_1 a^\phantom{\dagger}_1 -i\frac{\gamma}{2} a^\phantom{\dagger}_1 a^{\dagger}_1  + \omega_2 a^{\dagger}_2 a^\phantom{\dagger}_2 + J \left(a^{\dagger}_1 a^\phantom{\dagger}_2 + a^{\dagger}_2 a^\phantom{\dagger}_1\right)
$$

### Master equation:

$$
    \partial_{t} \hat{\rho}=-i\left[\hat{H} \hat{\rho}-\hat{\rho} \hat{H}^{
    \dagger}\right]+\lambda a^\phantom{\dagger}_1 \hat{\rho}  a^{\dagger}_1 +\gamma  a^{\dagger}_1 \hat{\rho} a^\phantom{\dagger}_1
$$

### Equations of motion

#### Vertical Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_t - \omega_1 + i(\lambda + \gamma)/2 & -J \\
    -J & i \partial_t - \omega_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t') 
    - i\gamma
    \begin{pmatrix}
    G^{\tilde{T}}_{11} & G^{\tilde{T}}_{12} \\
    0 & 0
    \end{pmatrix}(t, t')  \\
    0 &= \begin{pmatrix}
    i \partial_t - \omega_1 - i(\lambda + \gamma)/2 & -J \\
    -J & i \partial_t - \omega_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t')  
    + i\lambda
    \begin{pmatrix}
    G^T_{11} & G^T_{12} \\
    0 & 0
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Horizontal Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \omega_1 + i(\lambda + \gamma)/2 & J \\
    J & i \partial_{t'} + \omega_2
    \end{pmatrix} 
    - i\gamma
    \begin{pmatrix}
    G^{{T}}_{11} & 0 \\
    G^{{T}}_{21} & 0
    \end{pmatrix}(t, t')     \\
    0 &= \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \omega_1 - i(\lambda + \gamma)/2 & J \\
    J & i \partial_{t'} + \omega_2
    \end{pmatrix}  
    + i\lambda
    \begin{pmatrix}
    G^{\tilde{T}}_{11} & 0 \\
    G^{\tilde{T}}_{21} & 0
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Equal-Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_T + i(\lambda + \gamma) & 0 \\
    0 & i \partial_T
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)
    - \left[\begin{pmatrix}
    \omega_1 & J \\
    J & \omega_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)\right] 
    - i\gamma
    \begin{pmatrix}
    G^T_{11} + G^{\tilde{T}}_{11} & G^\tilde{T}_{12} \\
    G^{{T}}_{21} & 0
    \end{pmatrix}(T, 0)       \\
    0 &= \begin{pmatrix}
    i \partial_T - i(\lambda + \gamma) & 0 \\
    0 & i \partial_T
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)  
    - \left[\begin{pmatrix}
    \omega_1 & J \\
    J & \omega_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)\right]
    + i\lambda
    \begin{pmatrix}
    G^T_{11} + G^{\tilde{T}}_{11} & G^T_{12} \\
    G^{\tilde{T}}_{21} & 0
    \end{pmatrix}(T, 0)   
\end{split}\end{align}

## Solving

In [ ]:
# time parameters
T = 1.0

# Non-Hermitian Hamiltonian and jump operator
ω₁ = 0.1 * 5.0
ω₂ = 0.0
J = 0*0.5*2*pi*1.0/2 
U = 0*1.0
λ = 1.0
γ = 0*1.0
h = ComplexF64[ω₁ - 0.5im * (λ + γ) J; J ω₂];

In [ ]:
# quantum numbers
dim = 2

# Define your Green functions at (t0, t0), time-arguments at the end!
gfL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), Lesser)
gfG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), Greater)

# Note to Tim: Accessing a gf with 2 indices gives you the gf at those time points
# Lesser[1,1] <=> Lesser[:,:,1,1]

# initial condition
N_0 = 2

gfL[1,1,1,1] = -1.0im * N_0
gfG[1,1,1,1] = -1.0im * (N_0 + 1)
gfG[2,2,1,1] = -1.0im

u0 = [gfL, gfG];

function rhs_vert(u, times, t, t′)
    Gᴸ, Gᴳ = u[1], u[2]
    rhs_L = -1.0im * (h * Gᴸ[t,t′] + [[1.0im * γ, 0] [0, 0]] * Gᴸ[t,t′])
    rhs_G = -1.0im * (adjoint(h) * Gᴳ[t,t′] - [[1.0im * λ, 0] [0, 0]] * Gᴳ[t,t′])
  return [rhs_L, rhs_G]
end

function rhs_diag(u, times, t)
    println("t: $(times[t])")
    Gᴸ, Gᴳ = u[1], u[2]
    rhs_L = -1.0im * (h * Gᴸ[t,t] - Gᴸ[t,t] * adjoint(h)
        + 1.0im * γ * [[Gᴸ[1,1,t,t] + Gᴳ[1,1,t,t], 0.5 * (Gᴸ[2,1,t,t] + Gᴳ[2,1,t,t])] [0.5 * (Gᴸ[1,2,t,t] + Gᴳ[1,2,t,t]), 0]])
    rhs_G = -1.0im * (adjoint(h) * Gᴳ[t,t] - Gᴳ[t,t] * h
        - 1.0im * λ * [[Gᴸ[1,1,t,t] + Gᴳ[1,1,t,t], 0.5 * (Gᴸ[2,1,t,t] + Gᴳ[2,1,t,t])] [0.5 * (Gᴸ[1,2,t,t] + Gᴳ[1,2,t,t]), 0]])
  return [rhs_L, rhs_G]
end

# Basically this will mutate Lesser and Greater in place
sol, _ = kbsolve(rhs_vert, rhs_diag, u0, (0.0, T); dtini=1e-6, atol=1e-4, rtol=1e-4);

In [ ]:
# [(t, tp) for t in sol.t, tp in sol.t]

In [ ]:
# println(map(x -> round(x[2] - x[1], sigdigits=4), zip(sol.t[1:end-1], sol.t[2:end])))

In [ ]:
length(sol.t)

In [ ]:
# n = 128;
n = length(sol.t) - 1;

In [ ]:
stop = Int(round(n/2)) + 1 

times = sol.t # range(first(sol.t), stop=last(sol.t), length=n + 1) |> collect

horizontal_times = k -> vcat(times[k:end] .- times[k], times[end] .+ (1:(k - 1) |> collect) .* (times[end] - times[end-1]));

In [ ]:
stop

## QuTiP benchmark

In [ ]:
# maximal number of photons
# Fock-space dimension will be n_max + 1
n_max = 10

# initial state
psi0_list = [qt.basis(n_max + 1, N_0), qt.basis(n_max + 1, 0)]
psi0 = qt.tensor(psi0_list)

# define annihilation operators
a_1_list = [qt.destroy(n_max + 1), qt.qeye(n_max + 1)]
a_1  = qt.tensor(a_1_list)    
a_2_list = [qt.qeye(n_max + 1), qt.destroy(n_max + 1)]
a_2  = qt.tensor(a_2_list)    

# Hamiltonian
H  = J * a_1.dag() * a_2
H += H.dag()
H += ω₁ * a_1.dag() * a_1 + ω₂ * a_2.dag() * a_2
# H += 0.5 * U *(a_1.dag() * a_1 * a_1.dag() * a_1 + a_2.dag() * a_2 * a_2.dag() * a_2)

observables = [a_1.dag()*a_1, a_2.dag()*a_2, a_1*a_1.dag(), a_2*a_2.dag()];

In [ ]:
j_ops = [sqrt(λ) * a_1, sqrt(γ) * a_1.dag()] # qutip only needs the jump operator

# quickly solve once for observables
me = qt.mesolve(H, psi0, times, j_ops, observables)

# solve for the time-dependent density matrix
t_sols = qt.mesolve(H, psi0, times, j_ops); # t_sols.states returns density matrices

#### Two times

Calculate $\langle a^\dagger_1(t') a^\phantom{\dagger}_1(t)\rangle$, where $t' = t + \tau$.

In [ ]:
tau_t_sols = Dict()
for k in 1:length(t_sols.states)
    tau_t_sols[k] = qt.mesolve(H, a_1 * t_sols.states[k], horizontal_times(k), j_ops).states
end

a_1_dag_a_1 = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
for k in 1:length(t_sols.states)
    for l in 1:length(t_sols.states)
        a_1_dag_a_1[k, l] = (a_1.dag() * tau_t_sols[k][l]).tr()    
    end
end

In [ ]:
tau_t_sols = Dict()
for k in 1:length(t_sols.states)
    tau_t_sols[k] = qt.mesolve(H, a_1.dag() * t_sols.states[k], horizontal_times(k), j_ops).states
end

a_1_a_1_dag = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
for k in 1:length(t_sols.states)
    for l in 1:length(t_sols.states)
        a_1_a_1_dag[k, l] = (a_1 * tau_t_sols[k][l]).tr()    
    end
end

In [ ]:
# reshape the above array to fit into our two-time "matrix" structure 
# see the plot below for illustration
unskewed_a_1_dag_a_1 = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
for (k, x) in enumerate([a_1_dag_a_1[k, :] for k in 1:length(t_sols.states)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_a_1_dag_a_1[k, ind] = y  
    end
end

In [ ]:
# reshape the above array to fit into our two-time "matrix" structure 
# see the plot below for illustration
unskewed_a_1_a_1_dag = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
for (k, x) in enumerate([a_1_a_1_dag[k, :] for k in 1:length(t_sols.states)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_a_1_a_1_dag[k, ind] = y  
    end
end

In [ ]:
cmap = "gist_heat"

In [ ]:
figure(figsize=(6, 2))
subplot(121)
imshow(real(a_1_dag_a_1), cmap=cmap)

subplot(122)
imshow(real(unskewed_a_1_a_1_dag), cmap=cmap)
colorbar()

tight_layout()

In [ ]:
# figure(figsize=(8, 5))

# subplot(121)
# imshow(real(a_1_a_1_dag), cmap=cmap)

# subplot(122)
# imshow(, cmap=cmap)
# # colorbar()

Y, X = np.meshgrid(sol.t, sol.t);
rho_11 = imag(unskewed_a_1_a_1_dag - unskewed_a_1_dag_a_1)[:, 1:n+1]

figure(figsize=(5, 3))
ax = plt.gca()
heatmap = ax.pcolormesh(X, Y, (rho_11 |> transpose) .+ rho_11, cmap="gist_heat", vmin=minimum(rho_11), vmax=maximum(rho_11))
ax.set_aspect("equal")
colorbar(mappable=heatmap)
ax.set_xlabel("\$t\$")
ax.set_xlabel("\$t'\$")
# ax.legend()
tight_layout()

## Plotting

In [ ]:
a_1_dag_a_1_tau = [unskewed_a_1_dag_a_1[stop - (k - 1), stop + (k - 1)] for k in 1:stop]
a_1_dag_a_1_tau_reversed = [a_1_dag_a_1_tau[stop - (k - 1)] for k in 1:stop]
a_1_dag_a_1_tau_full = vcat(conj(a_1_dag_a_1_tau_reversed[1:end-1]), a_1_dag_a_1_tau);

a_2_dag_a_2_tau = [unskewed_a_2_dag_a_2[stop - (k - 1), stop + (k - 1)] for k in 1:stop]
a_2_dag_a_2_tau_reversed = [a_2_dag_a_2_tau[stop - (k - 1)] for k in 1:stop]
a_2_dag_a_2_tau_full = vcat(conj(a_2_dag_a_2_tau_reversed[1:end-1]), a_2_dag_a_2_tau);

In [ ]:
figure(figsize=(12, 6))
subplot(221)
# plot(sol.t, [-imag(gfL.data[1, 1, k, k]) for k in 1:length(sol.t)], ls="-", c="r")
# plot(sol.t, [-imag(gfL.data[2, 2, k, k]) for k in 1:length(sol.t)], ls="-", c="r")
# plot(times, me.expect[1], c="k", ls="--", lw=3.0, alpha=0.5)
plot(sol.t, [-imag(gfG.data[1, 1, k, k]) for k in 1:length(sol.t)], ls="-", c="r")
plot(times, [real(unskewed_a_1_a_1_dag[k, k]) for k in 1:n+1], c="k", ls="--", lw=3.0, alpha=0.5)
# plot(times, me.expect[2], c="k", ls="--")
xlim(0, T)
# ylim(0, N_0)
xlabel("\$T\$")

subplot(222)
idx = 1
# plot(sol.t[idx:end - 1], [-imag((gfL.data - gfG.data)[1, 1, idx, k + idx - 1]) for k in 1:length(sol.t) - idx], ls="-", c="r")
# plot(sol.t[idx:end - 1], [imag((unskewed_a_1_dag_a_1 - unskewed_a_1_a_1_dag)[idx, k + idx - 1]) .- 1 for k in 1:length(sol.t) - idx], c="k", ls="--", lw=3.0, alpha=0.5)

plot(sol.t[idx:end - 1], [-imag(gfL.data[1, 1, idx, k + idx - 1]) for k in 1:length(sol.t) - idx], ls="-", c="r")
plot(sol.t[idx:end - 1], [real(unskewed_a_1_dag_a_1[idx, k + idx - 1]) for k in 1:length(sol.t) - idx], c="k", ls="--", lw=3.0, alpha=0.5)


# xlim(-T, T)
xlabel("\$t\$")
# ylim(-5.0, 5.0)

subplot(223)
plot(sol.t[idx:end - 1], [-imag(gfG.data[1, 1, idx, k + idx - 1]) for k in 1:length(sol.t) - idx], ls="-", c="r")
plot(sol.t[idx:end - 1], [real(unskewed_a_1_a_1_dag[idx, k + idx - 1]) for k in 1:length(sol.t) - idx], c="k", ls="--", lw=3.0, alpha=0.5)

# xlim(0, T)
# # ylim(0, N_0)
# xlabel("\$t\$")

tight_layout()
# savefig("test.pdf")

In [ ]:
Y, X = np.meshgrid(sol.t, sol.t);
ρ_11 = imag.(gfL.data[1, 1, :, :] - gfG.data[1, 1, :, :]);

In [ ]:
figure(figsize=(5, 3))
ax = plt.gca()
heatmap = ax.pcolormesh(X, Y, ρ_11, cmap="gist_heat", vmin=minimum(ρ_11), vmax=maximum(ρ_11))
ax.set_aspect("equal")
colorbar(mappable=heatmap)
ax.set_xlabel("\$t\$")
ax.set_xlabel("\$t'\$")
# ax.legend()
tight_layout()

In [ ]:
figure(figsize=(12, 3))
subplot(121)
plot(sol.t, [-imag(gfL.data[1, 1, k, k]) for k in 1:length(sol.t)] .- me.expect[1], marker="", ms=3.0, ls="-", c="r")
# plot(times, GL_11 - me.expect[1], c="k", ls="--")
# plot(times, GL_22 - me.expect[2], c="k", ls="--")
# plot(times, GG_11 - me.expect[3], c="k", ls=":")
# plot(times, GG_22 - me.expect[4], c="k", ls=":")
xlim(0, T)
# ylim(0, N_0)
xlabel("\$t\$")

subplot(122)
idx = 20
plot(sol.t[idx:end - 1], [-imag(gfL.data[1, 1, idx, k + idx - 1]) for k in 1:length(sol.t) - idx] 
    .- [real(unskewed_a_1_dag_a_1[idx, k + idx - 1]) for k in 1:length(sol.t) - idx], "o", ms=1, c="r")
# plot(times_tau, -imag(GL_11_tau_full) - a_1_dag_a_1_tau_full, c="k", ls="--")
# plot(times_tau, -imag(GL_22_tau_full) - a_2_dag_a_2_tau_full, c="k", ls="--")
xlim(-T, T)
xlabel("\$\\tau\$")
# ylim(0, 1.0)

tight_layout()

### Analytical only works for closed system

In [ ]:
# Analytic result
horizontal_times = x -> vcat(sol.t[x:end] .- sol.t[x], sol.t[end] .+ (1:(x - 1) |> collect) .* (sol.t[end] - sol.t[end-1]));
f(T, tp) = exp(-1.0im * h * horizontal_times(T)[tp]) * exp(-1.0im * h * sol.t[T]) * gfL[:, :, 1, 1] * exp(1.0im * h * sol.t[T])
f_diag(T) = exp(-1.0im * h * sol.t[T]) * gfL[:, :, 1, 1] * exp(1.0im * h * sol.t[T])

In [ ]:
figure(figsize=(12, 6))
subplot(221)
plot(sol.t, [imag(gfL.data[1, 1, k, k]) for k in 1:length(sol.t)], marker="", ms=3.0, ls="-", c="r")
plot(sol.t, me.expect[1], c="k", ls="--", lw=3.0, alpha=0.5)
plot(sol.t, [real(unskewed_a_1_dag_a_1[k, k]) for k in 1:n+1], c="k", ls="--", lw=3.0, alpha=0.5)
plot(sol.t, [imag(f_diag(k)[1, 1]) for k in 1:n+1], c="C0", ls="--", lw=5.0, alpha=0.5)
xlim(0, T)
# ylim(0, N_0)
xlabel("\$t\$")

subplot(222)
idx = 18
# plot(sol.t[idx:end - 1], [real(unskewed_a_1_dag_a_1[idx, k + idx - 1]) for k in 1:length(sol.t) - idx], "o", c="k", lw=3.0, alpha=0.5, ms=3)
plot(sol.t[idx:end - 1], [imag(gfL.data[1, 1, idx, k + idx - 1]) for k in 1:length(sol.t) - idx], "o", ms=3, c="r")
plot(sol.t[idx:end - 1], [imag(f(idx, k))[1, 1] for k in 1:length(sol.t) - idx], "x", c="C0", lw=5.0, alpha=0.5, ms=5)

tight_layout()

In [ ]:
figure(figsize=(12, 6))
subplot(221)
plot(sol.t, [imag(gfL.data[1, 1, k, k]) for k in 1:length(sol.t)] 
    .- [imag(f_diag(k)[1, 1]) for k in 1:n+1], marker="", ms=3.0, ls="-", c="r")
# plot(sol.t, me.expect[1] .- [imag(f_diag(k)[1, 1]) for k in 1:n+1], c="k", ls="--", lw=3.0, alpha=0.5)
xlim(0, T)
# ylim(0, N_0)
xlabel("\$t\$")

subplot(222)
idx = 10
plot(sol.t[idx:end - 1], [imag(gfL.data[1, 1, idx, k + idx - 1]) for k in 1:length(sol.t) - idx]
    .- [imag(f(idx, k))[1, 1] for k in 1:length(sol.t) - idx], "o", ms=1, c="r")
# semilogy(sol.t[idx:end - 1], [real(unskewed_c_1_dag_c_1[idx, k + idx - 1]) for k in 1:length(sol.t) - idx] .- [imag(f(idx, k))[1, 1] for k in 1:length(sol.t) - idx], "o", c="k", lw=3.0, alpha=0.5, ms=1)

tight_layout()
# savefig("test.pdf")

## Testing

In [ ]:
epsilons = [2.0^(-k) for k in 10:55]
init_dt = 1e-16 # epsilons[end]
err_data = [(0.0, 0.0) for _ in 1:length(epsilons)]
for (k, eps) in enumerate(epsilons)
    print(k, ", ")
    
    sol, _ = kbsolve(rhs_vert, rhs_diag, u0, (0.0, T); dtini=init_dt, atol=eps, rtol=eps);
    n = length(sol.t) - 1;
    
    # Analytic result
    horizontal_times = x -> vcat(sol.t[x:end] .- sol.t[x], sol.t[end] .+ (1:(x - 1) |> collect) .* (sol.t[end] - sol.t[end-1]));
    f(T, tp) = exp(-1.0im * h * horizontal_times(T)[tp]) * exp(-1.0im * h * sol.t[T]) * gfL[:, :, 1, 1] * exp(1.0im * h * sol.t[T])
    f_diag(T) = exp(-1.0im * h * sol.t[T]) * gfL[:, :, 1, 1] * exp(1.0im * h * sol.t[T])
    
    horizontal_err = 0.0
    for idx in 1:n
        horizontal_err += norm([imag(gfL.data[1, 1, idx, k + idx]) for k in 1:length(sol.t) - idx] 
            .- [imag(f(idx, k))[1, 1] for k in 1:length(sol.t) - idx + 1][2:end]) # not counting the diagonal
    end    

    err = norm([imag(gfL.data[1, 1, k, k]) for k in 1:length(sol.t)] .- [imag(f_diag(k)[1, 1]) for k in 1:n+1])
    err += 2 * horizontal_err
    err_data[k] = (n, (1.0/(n)^2) * err)    
end

In [ ]:
figure(figsize=(8, 5))

subplot(221)
loglog([x[1] for x in err_data], [x[2] for x in err_data], "s", lw=0, ms=3)
axhline(init_dt, c="grey", ls="--", label="\$\\Delta t_{\\mathrm{init}}\$")
xlim(1e1, 1e3)
ylim(1e-17, 1e-0)
xlabel("\$n\$")
ylabel("\$\\varepsilon_{\\mathrm{abs}}\$")
legend()

subplot(222)
plot(epsilons, epsilons)
loglog(epsilons, [x[2] for x in err_data], "s", lw=0, ms=3, alpha=0.5)
axhline(init_dt, c="grey", ls="--", label="\$\\Delta t_{\\mathrm{init}}\$")
xlim(1e-18, 1e-1)
ylim(1e-17, 1e-0)
yticks([])
xlabel("\$\\mathrm{atol}=\\mathrm{rtol}\$")
# ylabel("\$\\varepsilon_{\\mathrm{abs}}\$")

tight_layout()

In [ ]:
idx_1 = 20
plot(sol.t, [-imag(gfL.data[1, 1, idx_1, k]) for k in 1:n+1], "o", ms=3.0, c="r")
plot(sol.t, [a_1_dag_a_1[idx_1, k] for k in 1:n+1], c="k", "s", lw=3.0, alpha=0.5, ms=2)

In [ ]:
[wigner_transform(gfL.data[1, 1, :, :]; fourier=false)[1] for k in length(sol.t)]

In [ ]:
figure(figsize=(12, 3))
subplot(121)
xlim(0, T)
plot(sol.t, [-imag(gfL.data[1, 1, k, k]) for k in 1:length(sol.t)], marker="o", ms=3.0, ls="")
plot(sol.t, [-imag(gfG.data[1, 1, k, k]) for k in 1:length(sol.t)] .- 1, marker="x", ms=4.0, ls="")
tight_layout()

In [ ]:
figure(figsize=(12, 3))
subplot(121)
xlim(0, T/2)
plot(sol.t, [-imag(gfL.data[1, 1, k, k]) for k in 1:length(sol.t)] .- ([-imag(gfG.data[1, 1, k, k]) for k in 1:length(sol.t)] .- 1), marker="o", ms=3.0, ls="")
tight_layout()